In [1]:
%run ./model_end_events_for_outages_METHODS.ipynb

In [2]:
from importlib import reload
#reload(Utilities)
#reload(clm)
# NOTE: To reload a class imported as, e.g., 
# from module import class
# One must call:
#   1. import module
#   2. reload module
#   3. from module import class

#---------------------------------------------------------------------
import sys, os
import re
from pathlib import Path
import json
import pickle
import joblib

import pandas as pd
import numpy as np
from pandas.api.types import is_numeric_dtype, is_datetime64_dtype, is_timedelta64_dtype
from scipy import stats
import datetime
import time
from natsort import natsorted, ns, natsort_keygen
from packaging import version
import random

import copy
import itertools
import adjustText

import pyodbc
#---------------------------------------------------------------------
import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns
from matplotlib.backends.backend_pdf import PdfPages
import matplotlib.patches as mpatches
from matplotlib.lines import Line2D
import matplotlib.ticker as ticker
from matplotlib import dates
import matplotlib.colors as mcolors
import matplotlib.cm as cm #e.g. for cmap=cm.jet
#---------------------------------------------------------------------
sys.path.insert(0, os.path.realpath('..'))
import Utilities_config
#-----
from MeterPremise import MeterPremise
from EEMSP import EEMSP
#-----
from AMI_SQL import AMI_SQL
from AMINonVee_SQL import AMINonVee_SQL
from AMIEndEvents_SQL import AMIEndEvents_SQL
from AMIUsgInst_SQL import AMIUsgInst_SQL
from DOVSOutages_SQL import DOVSOutages_SQL
#-----
from GenAn import GenAn
from AMINonVee import AMINonVee
from AMIEndEvents import AMIEndEvents
from MECPODf import MECPODf
from MECPOAn import MECPOAn
from AMIUsgInst import AMIUsgInst
from DOVSOutages import DOVSOutages
from OutagePredictor import OutagePredictor
#---------------------------------------------------------------------
sys.path.insert(0, Utilities_config.get_sql_aids_dir())
import Utilities_sql
import TableInfos
from TableInfos import TableInfo
from SQLElement import SQLElement
from SQLElementsCollection import SQLElementsCollection
from SQLSelect import SQLSelectElement, SQLSelect
from SQLFrom import SQLFrom
from SQLWhere import SQLWhereElement, SQLWhere
from SQLJoin import SQLJoin, SQLJoinCollection
from SQLGroupBy import SQLGroupByElement, SQLGroupBy
from SQLHaving import SQLHaving
from SQLOrderBy import SQLOrderByElement, SQLOrderBy
from SQLQuery import SQLQuery
from SQLQueryGeneric import SQLQueryGeneric
#---------------------------------------------------------------------
#sys.path.insert(0, os.path.join(os.path.realpath('..'), 'Utilities'))
sys.path.insert(0, Utilities_config.get_utilities_dir())
import Utilities
import Utilities_df
from Utilities_df import DFConstructType
import Utilities_dt
import Plot_General
import Plot_Box_sns
import Plot_Hist
import Plot_Bar
import GrubbsTest
import DataFrameSubsetSlicer
from DataFrameSubsetSlicer import DataFrameSubsetSlicer as DFSlicer
#---------------------------------------------------------------------

In [3]:
cbus_only = True

In [4]:
if cbus_only:
    trsf_pole_df = pd.read_pickle(r'C:\Users\s346557\Documents\LocalData\dovs_and_end_events_data\20240906\20230401_20240831\Outages\trsf_location_info_df.pkl')
    trsf_pole_nbs_to_ignore = [' ', 'TRANSMISSION', 'PRIMARY', 'NETWORK']
    #-------------------------
    cbus_zips = [
        43085, 
        43201, 
        43202, 
        43203, 
        43204, 
        43205, 
        43206, 
        43207, 
        43209, 
        43210, 
        43211, 
        43212, 
        43213, 
        43214, 
        43215, 
        43216, 
        43217, 
        43218, 
        43219, 
        43220, 
        43221, 
        43222, 
        43223, 
        43224, 
        43226, 
        43227, 
        43228, 
        43229, 
        43230, 
        43231, 
        43232, 
        43234, 
        43235, 
        43236, 
        43240
    ] 
    
    #----------------------------------------------------------------------------------------------------
    trsf_pole_df = trsf_pole_df.drop_duplicates()
    trsf_pole_df = trsf_pole_df.reset_index(drop=True)
    trsf_pole_df = trsf_pole_df[~trsf_pole_df['trsf_pole_nb'].isin(trsf_pole_nbs_to_ignore)]
    #-------------------------
    # Want only one entry per zip
    # For each trsf_pole_nb, keep only the entry with the largest nPNs value
    trsf_pole_df = trsf_pole_df.reset_index(drop=True)
    trsf_pole_df = trsf_pole_df.loc[trsf_pole_df.groupby('trsf_pole_nb')['nPNs_xfmri_sa4i'].idxmax()]
    trsf_pole_df = trsf_pole_df.reset_index(drop=True)
    assert(trsf_pole_df.shape[0]==trsf_pole_df['trsf_pole_nb'].nunique())
    #-------------------------
    trsf_pole_df['zip'] = trsf_pole_df['zip'].astype(int)
    trsf_pole_df_cbus = trsf_pole_df[trsf_pole_df['zip'].isin(cbus_zips)]

In [5]:
# prediction_date   = pd.to_datetime('2023-07-27')
# #-----
# model_base_dir = r'C:\Users\s346557\Documents\LocalData\dovs_and_end_events_data\20240906\Models\All_EEMSP_agg_Top10'
# #-----
# tmp_f = open(r'C:\Users\s346557\Documents\LocalData\dovs_and_end_events_data\20240906\Models\All_EEMSP_agg_Top10\Slices\ledger.json')
# ledger = json.load(tmp_f)
# tmp_f.close()
# trsf_pole_nbs_model = natsorted(ledger.keys())
# #-----
# random.seed(42)
# #-----
# n_poles = 100
# if cbus_only:
#     trsf_pole_nbs_init = list(set(trsf_pole_df_cbus['trsf_pole_nb'].unique()).intersection(set(trsf_pole_nbs_model)))
#     trsf_pole_nbs_init = random.sample(trsf_pole_nbs_init,  n_poles)
# else:
#     trsf_pole_nbs_init = random.sample(trsf_pole_nbs_model, n_poles)

In [6]:
prediction_date   = pd.to_datetime('2023-07-27')
#-----
model_base_dir = r'C:\Users\s346557\Documents\LocalData\dovs_and_end_events_data\20240906\Models\All_EEMSP_agg_Top10_noMonth_20250219'
#-----
tmp_f = open(r'C:\Users\s346557\Documents\LocalData\dovs_and_end_events_data\20240906\Models\All_EEMSP_agg_Top10_noMonth_20250219\Slices\ledger.json')
ledger = json.load(tmp_f)
tmp_f.close()
#-----
random.seed(42)

In [7]:
assert(0)

AssertionError: 

In [8]:
self = OutagePredictor(prediction_date=prediction_date)
#-----
self.set_model_dir(
    model_dir             = model_base_dir, 
    model_by_zip_clusters = True, 
)
#-----
self.set_trsf_pole_nbs_from_sql(
    n_trsf_pole_nbs = 100, 
    states          = 'OH', 
    city            = 'COLUMBUS'
)
print(f"n_trsf_pole_nbs = {len(self.trsf_pole_nbs)}")
print(f"\nSQL statement:\n{self.trsf_pole_nbs_sql}")
#-----
self.initialize_data(
    evsSum_sql_fcn    = AMIEndEvents_SQL.build_sql_end_events,  
    evsSum_sql_kwargs = dict(opco='oh'), 
)

n_trsf_pole_nbs = 100

SQL statement:
WITH Ohb AS (
	SELECT DISTINCT
		MP.trsf_pole_nb
	FROM default.meter_premise MP
	WHERE MP.state_cd = 'OH'
	AND   MP.serv_city_ad = 'COLUMBUS'
)
SELECT *
FROM Ohb
ORDER BY RAND()
LIMIT 100
n_coll     = 100
batch_size = 1000
n_batches  = 1
trsf_pole_zips_df build time: 0.0162203311920166
n_coll     = 100
batch_size = 1000
n_batches  = 1
n_coll     = 100
batch_size = 10000
n_batches  = 1
!!!!! WARNING !!!!! NO OPCOs SELECTED!


In [13]:
self.predict()
y_scores_df = self.y_pred_df.copy()

In [15]:
y_scores_df['pred'].sum()

3

In [17]:
y_scores_df = pd.merge(
    y_scores_df, 
    self.rcpx_df['EEMSP_0'], 
    how         = 'inner', 
    left_index  = True, 
    right_index = True
)
y_scores_df = y_scores_df.sort_values(by=['prob_1'], ascending=False)

In [54]:
self.zip_clstrs_ledger

True

In [ ]:
self.xfmrs_ledger

In [ ]:
assert(0)

In [64]:
# model_i = 7
model_i = 0
trsf_poles_i = self.xfmrs_ledger[model_i]

print(model_i)
model_i_dir = os.path.join(model_base_dir, 'Slices', str(model_i))
assert(os.path.isdir(model_i_dir))
#-------------------------
# If no model able to be made for group, directory will be empty
# if not os.listdir(model_i_dir):
#     print('No model compiled.\nContinuing to next')
#     continue
#-------------------------
predictor_i = OutagePredictor(prediction_date=prediction_date)
predictor_i.set_model_dir(model_i_dir)
predictor_i.set_trsf_pole_nbs(trsf_poles_i)
predictor_i.initialize_data(
    evsSum_sql_fcn=AMIEndEvents_SQL.build_sql_end_events,  
    evsSum_sql_kwargs=dict(opco='oh'), 
)
#-------------------------
# assert(model_i not in predictors.keys())
# predictors[model_i] = predictor_i

0
n_coll     = 8
batch_size = 1000
n_batches  = 1
n_coll     = 8
batch_size = 10000
n_batches  = 1
!!!!! WARNING !!!!! NO OPCOs SELECTED!


In [65]:
len(list(set(predictor_i.trsf_pole_nbs).difference(set(predictor_i.evsSum_df['trsf_pole_nb'].unique()))))

0

In [66]:
list(set(predictor_i.trsf_pole_nbs).difference(set(predictor_i.evsSum_df['trsf_pole_nb'].unique())))

[]

In [70]:
dovs_df, mp_df = self.build_dovs_df(return_mp_df=True)

n_coll     = 100
batch_size = 1000
n_batches  = 1
n_coll     = 623
batch_size = 1000
n_batches  = 1
n_coll     = 620
batch_size = 1000
n_batches  = 1
!!!!! WARNING !!!!! NO OPCOs SELECTED!


In [71]:
dovs_df['trsf_pole_nb'].nunique()

11

In [72]:
y_scores_df[y_scores_df['pred']==1].index.nunique()

3

In [73]:
set(dovs_df['trsf_pole_nb'].unique()).intersection(set(y_scores_df[y_scores_df['pred']==1].index.unique()))

set()

In [84]:
prediction_dates = pd.date_range(
    start = pd.to_datetime('2023-09-01'), 
    end   = pd.to_datetime('2023-09-03'), 
    freq  = pd.Timedelta('1D')
)
prediction_dates

DatetimeIndex(['2023-09-01', '2023-09-02', '2023-09-03'], dtype='datetime64[ns]', freq='D')

In [86]:
outg_prdctr = OutagePredictor(prediction_date=prediction_dates[0])
#-----
outg_prdctr.set_model_dir(r'C:\Users\s346557\Documents\LocalData\dovs_and_end_events_data\20231201\Models\All_EEMSP_agg_Top10_v5')
outg_prdctr.set_trsf_pole_nbs(trsf_pole_nbs[:10])

outg_prdctr.prep_multiple_prediction_dates(
    prediction_dates  = prediction_dates, 
    predictions_range = None, 
    evsSum_sql_fcn    = AMIEndEvents_SQL.build_sql_end_events,  
    evsSum_sql_kwargs = dict(opco='oh'), 
    save_args         = False
)

n_coll     = 10
batch_size = 1000
n_batches  = 1


In [87]:
outg_prdctr.date_range

[datetime.date(2023, 8, 1), datetime.date(2023, 8, 31)]

In [88]:
outg_prdctr.daq_date_ranges

[(datetime.date(2023, 8, 1), datetime.date(2023, 9, 2))]

In [89]:
outg_prdctr.evsSum_df['trsf_pole_nb'].nunique()

9

In [90]:
len(trsf_pole_nbs)

100

In [91]:
date_col='aep_event_dt'
trsf_pole_nb_col  = 'trsf_pole_nb'
self = outg_prdctr

In [93]:
assert(0)

AssertionError: 

In [94]:
predictors = dict()
for model_i, trsf_poles_i in ledger_inv_xfmr.items():
    print(model_i)
    model_i_dir = os.path.join(model_base_dir, 'Slices', str(model_i))
    assert(os.path.isdir(model_i_dir))
    #-------------------------
    # If no model able to be made for group, directory will be empty
    if not os.listdir(model_i_dir):
        print('No model compiled.\nContinuing to next')
        continue
    #-------------------------
    predictor_i = OutagePredictor(prediction_date=prediction_date)
    predictor_i.set_model_dir(model_i_dir)
    predictor_i.set_trsf_pole_nbs(trsf_poles_i)
    predictor_i.initialize_data(
        evsSum_sql_fcn=AMIEndEvents_SQL.build_sql_end_events,  
        evsSum_sql_kwargs=dict(opco='oh'), 
    )
    #-------------------------
    assert(model_i not in predictors.keys())
    predictors[model_i] = predictor_i

0
n_coll     = 8
batch_size = 1000
n_batches  = 1
n_coll     = 8
batch_size = 10000
n_batches  = 1
!!!!! WARNING !!!!! NO OPCOs SELECTED!
9
n_coll     = 1
batch_size = 1000
n_batches  = 1
n_coll     = 1
batch_size = 10000
n_batches  = 1
!!!!! WARNING !!!!! NO OPCOs SELECTED!
16
n_coll     = 21
batch_size = 1000
n_batches  = 1
n_coll     = 21
batch_size = 10000
n_batches  = 1
!!!!! WARNING !!!!! NO OPCOs SELECTED!
19
n_coll     = 16
batch_size = 1000
n_batches  = 1
n_coll     = 16
batch_size = 10000
n_batches  = 1
!!!!! WARNING !!!!! NO OPCOs SELECTED!
24
n_coll     = 6
batch_size = 1000
n_batches  = 1
n_coll     = 6
batch_size = 10000
n_batches  = 1
!!!!! WARNING !!!!! NO OPCOs SELECTED!
26
n_coll     = 14
batch_size = 1000
n_batches  = 1
n_coll     = 14
batch_size = 10000
n_batches  = 1
!!!!! WARNING !!!!! NO OPCOs SELECTED!
33
n_coll     = 11
batch_size = 1000
n_batches  = 1
n_coll     = 11
batch_size = 10000
n_batches  = 1
!!!!! WARNING !!!!! NO OPCOs SELECTED!
43
n_coll     = 27
ba

In [95]:
for model_i, pred_i in predictors.items():
    y_pred_i = pred_i.model_clf.predict(pred_i.X_test)
    print(model_i)
    print(y_pred_i.sum())
    print()

0
0

9
0

16
1

19
0

24
0

26
0

33
1

43
3



In [98]:
pred_0 = predictors[0]
y_pred_0 = pred_0.model_clf.predict(pred_0.X_test)

In [100]:
y_pred_0.sum()

0

In [102]:
y_pred_0

array([0, 0, 0, 0, 0, 0, 0, 0], dtype=int64)

In [104]:
pred_0.rcpx_df

01-06 Days  \
              Detected end of voltage sag   
trsf_pole_nb                                
1814220722502                         0.0   
1814452722480                         0.0   
1817115711501                         0.4   
1822111734903                         0.4   
1824912721355                         0.0   
1825462723107                         0.0   
1832391711296                         0.0   
1834364723060                         0.0   

                                                                                           \
              Error occurred when attempting to synch meter time with NIC time for device   
trsf_pole_nb                                                                                
1814220722502                                                0.0                            
1814452722480                                                0.0                            
1817115711501                                                0.0                            
1822111734903                                                0.0                            
1824912721355                                                0.0                            
1825462723107                                                0.0                            
1832391711296                                                0.0                            
1834364723060                                                0.0                            

                                                                 \
              Primary Power Down Primary Power Up Under Voltage   
trsf_pole_nb                                                      
1814220722502                0.4              0.4           0.0   
1814452722480                0.4              0.4           0.0   
1817115711501                0.0              0.0           0.0   
1822111734903                0.0              0.0           0.0   
1824912721355                0.2              0.2           0.0   
1825462723107                0.2              0.2           0.0   
1832391711296                0.2              0.2           0.0   
1834364723060                0.2              0.2           0.0   

                                                     \
              NET_MGMT command failed consecutively   
trsf_pole_nb                                          
1814220722502                              0.000000   
1814452722480                              0.200000   
1817115711501                              0.000000   
1822111734903                              0.000000   
1824912721355                              0.120000   
1825462723107                              0.000000   
1832391711296                              0.014286   
1834364723060                              0.100000   

                                                           \
              NIC Power Restore Trap Received from device   
trsf_pole_nb                                                
1814220722502                                         0.0   
1814452722480                                         0.0   
1817115711501                                         0.0   
1822111734903                                         0.0   
1824912721355                                         0.0   
1825462723107                                         0.0   
1832391711296                                         0.0   
1834364723060                                         0.0   

                                                                    ...  \
              Detected end of voltage swell Last Gasp Over Voltage  ...   
trsf_pole_nb                                                        ...   
1814220722502                           0.0       0.0          0.0  ...   
1814452722480                           0.0       0.0          0.0  ...   
1817115711501                           0.0       0.0          0.0  ...   
1822111734903                           0.0  

In [106]:
conn_outages = Utilities.get_utldb01p_oracle_connection()

In [107]:
sql_outage_location_ids = DOVSOutages_SQL.build_sql_find_outage_xfmrs(
    mjr_mnr_cause   = None, 
    include_premise = False, 
    date_range      = [dt_0, dt_1], 
    states          = ['OH']
)
sql_stmnt_outage_location_ids = sql_outage_location_ids.get_sql_statement()
#-------------------------
df_outage_location_ids = pd.read_sql_query(
    sql_stmnt_outage_location_ids, 
    conn_outages
)

NameError: name 'dt_0' is not defined

In [ ]:
df_outage_location_ids

In [ ]:
#FUCK

In [ ]:
assert(0)

In [ ]:
trsf_pole_nb_col               = 'trsf_pole_nb'
t_min_col                      = 't_min'
t_max_col                      = 't_max'
trsf_pole_nbs_to_ignore        = [' ', 'TRANSMISSION', 'PRIMARY', 'NETWORK']
include_zero_counts            = True
build_sql_outage_kwargs        = None
assert_all_trsf_pole_nbs_found = False
save_args                      = False
batch_size                     = 30
n_update                       = 1
verbose                        = False

In [ ]:
# df          = pd.DataFrame(data=trsf_pole_nbs, columns=['trsf_pole_nb'])
# df['t_min'] = date_0
# df['t_max'] = date_1

prediction_date = pd.to_datetime('2024-07-27')
search_times_df = pd.DataFrame(
    data    = trsf_pole_df_cbus['trsf_pole_nb'].unique().tolist(), 
    columns = ['trsf_pole_nb']
)
search_times_df['t_min'] = prediction_date - pd.Timedelta('365D')
search_times_df['t_max'] = prediction_date - pd.Timedelta('1D')

In [ ]:
# search_times_df = search_times_df.iloc[:1000]
# search_times_df = search_times_df.iloc[:10]

In [ ]:
# n_outgs_per_xfmr_v1, n_outgs_per_xfmr_df_v1 = DOVSOutages.get_n_outgs_per_xfmr(
#     date_0                  = prediction_date - pd.Timedelta('365D'), 
#     date_1                  = prediction_date - pd.Timedelta('1D'), 
#     trsf_pole_nbs           = search_times_df['trsf_pole_nb'].unique().tolist(), 
#     include_zero_counts     = True, 
#     build_sql_outage_kwargs = None, 
#     save_args               = False, 
#     verbose                 = False, 
#     return_df               = True
# )

# n_outgs_per_xfmr_v1.to_pickle(r'C:\Users\s346557\Downloads\n_outgs_per_xfmr_v1.pkl')
# n_outgs_per_xfmr_df_v1.to_pickle(r'C:\Users\s346557\Downloads\n_outgs_per_xfmr_df_v1.pkl')

In [ ]:
# n_outgs_per_xfmr_v3, n_outgs_per_xfmr_df_v3 = DOVSOutages.get_n_outgs_per_xfmr_in_df(
#     search_times_df                = search_times_df, 
#     trsf_pole_nb_col               = trsf_pole_nb_col, 
#     t_min_col                      = t_min_col, 
#     t_max_col                      = t_max_col, 
#     trsf_pole_nbs_to_ignore        = trsf_pole_nbs_to_ignore, 
#     include_zero_counts            = True, 
#     build_sql_outage_kwargs        = build_sql_outage_kwargs, 
#     assert_all_trsf_pole_nbs_found = assert_all_trsf_pole_nbs_found, 
#     save_args                      = save_args, 
#     batch_size                     = batch_size, 
#     n_update                       = n_update, 
#     verbose                        = verbose, 
#     return_df                      = True
# )

# n_outgs_per_xfmr_v3.to_pickle(r'C:\Users\s346557\Downloads\n_outgs_per_xfmr_v3.pkl')
# n_outgs_per_xfmr_df_v3.to_pickle(r'C:\Users\s346557\Downloads\n_outgs_per_xfmr_df_v3.pkl')

In [ ]:
n_outgs_per_xfmr_v1 = pd.read_pickle(r'C:\Users\s346557\Downloads\n_outgs_per_xfmr_v1.pkl')
n_outgs_per_xfmr_df_v1 = pd.read_pickle(r'C:\Users\s346557\Downloads\n_outgs_per_xfmr_df_v1.pkl')

n_outgs_per_xfmr_v3 = pd.read_pickle(r'C:\Users\s346557\Downloads\n_outgs_per_xfmr_v3.pkl')
n_outgs_per_xfmr_df_v3 = pd.read_pickle(r'C:\Users\s346557\Downloads\n_outgs_per_xfmr_df_v3.pkl')

In [ ]:
n_outgs_per_xfmr_v3.droplevel(level=['t_min', 't_max']).equals(n_outgs_per_xfmr_v1)

In [ ]:
n_outgs_per_xfmr_v1.isna().sum()

In [ ]:
(n_outgs_per_xfmr_v1==0).sum()

In [ ]:
# n_outgs_per_xfmr_v1_wo0, n_outgs_per_xfmr_df_v1_wo0 = DOVSOutages.get_n_outgs_per_xfmr(
#     date_0                  = prediction_date - pd.Timedelta('365D'), 
#     date_1                  = prediction_date - pd.Timedelta('1D'), 
#     trsf_pole_nbs           = search_times_df['trsf_pole_nb'].unique().tolist(), 
#     include_zero_counts     = False, 
#     build_sql_outage_kwargs = None, 
#     save_args               = False, 
#     verbose                 = False, 
#     return_df               = True
# )

# n_outgs_per_xfmr_v1_wo0.to_pickle(r'C:\Users\s346557\Downloads\n_outgs_per_xfmr_v1_wo0.pkl')
# n_outgs_per_xfmr_df_v1_wo0.to_pickle(r'C:\Users\s346557\Downloads\n_outgs_per_xfmr_df_v1_wo0.pkl')

In [ ]:
# n_outgs_per_xfmr_v3_wo0, n_outgs_per_xfmr_df_v3_wo0 = DOVSOutages.get_n_outgs_per_xfmr_in_df(
#     search_times_df                = search_times_df, 
#     trsf_pole_nb_col               = trsf_pole_nb_col, 
#     t_min_col                      = t_min_col, 
#     t_max_col                      = t_max_col, 
#     trsf_pole_nbs_to_ignore        = trsf_pole_nbs_to_ignore, 
#     include_zero_counts            = False, 
#     build_sql_outage_kwargs        = build_sql_outage_kwargs, 
#     assert_all_trsf_pole_nbs_found = assert_all_trsf_pole_nbs_found, 
#     save_args                      = save_args, 
#     batch_size                     = batch_size, 
#     n_update                       = n_update, 
#     verbose                        = verbose, 
#     return_df                      = True
# )

# n_outgs_per_xfmr_v3_wo0.to_pickle(r'C:\Users\s346557\Downloads\n_outgs_per_xfmr_v3_wo0.pkl')
# n_outgs_per_xfmr_df_v3_wo0.to_pickle(r'C:\Users\s346557\Downloads\n_outgs_per_xfmr_df_v3_wo0.pkl')

In [ ]:
n_outgs_per_xfmr_v1_wo0 = pd.read_pickle(r'C:\Users\s346557\Downloads\n_outgs_per_xfmr_v1_wo0.pkl')
n_outgs_per_xfmr_df_v1_wo0 = pd.read_pickle(r'C:\Users\s346557\Downloads\n_outgs_per_xfmr_df_v1_wo0.pkl')

n_outgs_per_xfmr_v3_wo0 = pd.read_pickle(r'C:\Users\s346557\Downloads\n_outgs_per_xfmr_v3_wo0.pkl')
n_outgs_per_xfmr_df_v3_wo0 = pd.read_pickle(r'C:\Users\s346557\Downloads\n_outgs_per_xfmr_df_v3_wo0.pkl')

In [ ]:
n_outgs_per_xfmr_v3_wo0.droplevel(level=['t_min', 't_max']).sort_index().equals(n_outgs_per_xfmr_v1_wo0.sort_index())

In [ ]:
n_outgs_per_xfmr_v1_wo0.isna().sum()

In [ ]:
(n_outgs_per_xfmr_v1_wo0==0).sum()

In [ ]:
set(['43002', '43016', '43017', '43064', '43065', '43235']).intersection(set([str(x) for x in cbus_zips]))